# Association Rule Based Recommender System

## İş Problemi

* Türkiye’nin en büyük online hizmet platformu olan Armut, hizmet verenler ile hizmet almak isteyenleri buluşturmaktadır. Bilgisayarın veya akıllı telefonunun üzerinden birkaç dokunuşla temizlik, tadilat, nakliyat gibi hizmetlere kolayca ulaşılmasını sağlamaktadır. Hizmet alan kullanıcıları ve bu kullanıcıların almış oldukları servis ve kategorileri içeren veri setini kullanarak Association Rule Learning ile ürün tavsiye sistemi oluşturulmak istenmektedir.

## Veri Setinin Hikayesi
* Veri seti müşterilerin aldıkları servislerden ve bu servislerin kategorilerinden oluşmaktadır. Alınan her hizmetin tarih ve saat bilgisini içermektedir.

## Değişkenler
* UserId: Müşteri numarası
* ServiceId: Her kategoriye ait anonimleştirilmiş servislerdir. (Örnek : Temizlik kategorisi altında koltuk yıkama servisi)
* Bir ServiceId farklı kategoriler altında bulanabilir ve farklı kategoriler altında farklı servisleri ifade eder.(Örnek: CategoryId’si 7 ServiceId’si 4 olan hizmet petek temizliği iken CategoryId’si 2 ServiceId’si 4 olan hizmet mobilya montaj)
* CategoryId: Anonimleştirilmiş kategorilerdir. (Örnek : Temizlik, nakliyat, tadilat kategorisi)
* CreateDate: Hizmetin satın alındığı tarih

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [4]:
data = pd.read_csv("armut_data.csv")
print(data.head())
print(data.dtypes)

   UserId  ServiceId  CategoryId           CreateDate
0   25446          4           5  2017-08-06 16:11:00
1   22948         48           5  2017-08-06 16:12:00
2   10618          0           8  2017-08-06 16:13:00
3    7256          9           4  2017-08-06 16:14:00
4   25446         48           5  2017-08-06 16:16:00
UserId         int64
ServiceId      int64
CategoryId     int64
CreateDate    object
dtype: object


### Adım 1) ServiceID, her bir CategoryID özelinde farklı bir hizmeti temsil etmektedir. Bu iki değişkeni birleştirerek yeni bir değişken oluşturup dataframe'e ekleyelim. Bu sayede bir ARL recommender sistem oluşturmaya başlayabiliriz.

In [5]:
data["Hizmet"] = (data["ServiceId"].astype(str)) + "_" + (data["CategoryId"].astype(str))
data.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet
0,25446,4,5,2017-08-06 16:11:00,4_5
1,22948,48,5,2017-08-06 16:12:00,48_5
2,10618,0,8,2017-08-06 16:13:00,0_8
3,7256,9,4,2017-08-06 16:14:00,9_4
4,25446,48,5,2017-08-06 16:16:00,48_5


### Adım 2) Bir sepet oluşturalım. Sepet, her bir müşterinin aylık aldığı hizmetlerdir. Sepet içerisinede hizmetin alındığı ay ve yılı bu yüzden girmemiz gerekiyor.

In [6]:
data["CreateDate"] = pd.to_datetime(data["CreateDate"].astype(str), format = "%Y-%m-%d %H:%M:%S")
data["New_Date"] = data["CreateDate"].dt.strftime("%m-%Y")
data["SepetId"] = data["UserId"].astype(str) + "_" + data["New_Date"]
data.head()

,UserId,ServiceId,CategoryId,CreateDate,Hizmet,New_Date,SepetId
0,25446,4,5,2017-08-06 16:11:00,4_5,08-2017,25446_08-2017
1,22948,48,5,2017-08-06 16:12:00,48_5,08-2017,22948_08-2017
2,10618,0,8,2017-08-06 16:13:00,0_8,08-2017,10618_08-2017
3,7256,9,4,2017-08-06 16:14:00,9_4,08-2017,7256_08-2017
4,25446,48,5,2017-08-06 16:16:00,48_5,08-2017,25446_08-2017


### Adım 3) Birliktelik Kurallarını oluşturmadan önce, Sepet, hizmet pivot table'ı oluşturalım.

In [7]:
data_pt= data.groupby(["SepetId","Hizmet"])["Hizmet"].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)
print(data_pt)

/var/folders/l2/85cfp5d92ybfdr50zh9xvqsr0000gn/T/ipykernel_93585/3987805613.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  data_pt= data.groupby(["SepetId","Hizmet"])["Hizmet"].count().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0)


Hizmet         0_8  10_9  11_11  12_7  13_11  14_7  15_1  16_8  17_5  18_4  \
SepetId                                                                      
0_01-2018        0     0      0     0      0     0     0     0     0     0   
0_04-2018        0     0      0     0      0     1     0     0     0     0   
0_08-2017        0     0      0     0      0     0     0     0     0     0   
0_09-2017        0     0      0     0      0     0     0     0     0     0   
10000_03-2018    0     0      0     0      0     0     0     0     0     0   
...            ...   ...    ...   ...    ...   ...   ...   ...   ...   ...   
99_01-2018       1     0      0     0      0     0     0     0     0     0   
99_02-2018       1     0      0     0      0     0     0     0     0     0   
99_12-2017       1     0      0     0      0     0     0     0     0     0   
9_03-2018        0     0      0     0      0     0     0     0     0     0   
9_04-2018        0     0      0     0      0     0     0     0  

### Adım 4) Birliktelik Kurallarını oluşturalım

In [12]:
from mlxtend.frequent_patterns import apriori, association_rules

frequent_itemsets = apriori(data_pt,min_support=0.01,use_colnames=True)
rules = association_rules(frequent_itemsets,metric="support",min_threshold=0.01)
print(rules)

/Users/hasangulec/anaconda3/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


   antecedents consequents  antecedent support  consequent support   support  \
0        (2_0)     (13_11)            0.130286            0.056627  0.012819   
1      (13_11)       (2_0)            0.056627            0.130286  0.012819   
2        (2_0)      (15_1)            0.130286            0.120963  0.033951   
3       (15_1)       (2_0)            0.120963            0.130286  0.033951   
4       (33_4)      (15_1)            0.027310            0.120963  0.011233   
5       (15_1)      (33_4)            0.120963            0.027310  0.011233   
6       (38_4)      (15_1)            0.066568            0.120963  0.011177   
7       (15_1)      (38_4)            0.120963            0.066568  0.011177   
8       (49_1)      (15_1)            0.067762            0.120963  0.010011   
9       (15_1)      (49_1)            0.120963            0.067762  0.010011   
10      (25_0)      (22_0)            0.042895            0.047515  0.011120   
11      (22_0)      (25_0)            0.

* antecedents: x ürün, 
* consequents: y ürün, 
* antecedent support: x ürünün tek basına gözlenme olasılığı, 
* consequent support: y ürünün tek basına gözlenme olasılığı
* support : ikisinin birlikte gözlenme olasılığı, 
* confidince : x satın alındığında y nin satın alınma olasılığı
* lift : x satın alındığında y'nin satın alınma olasılığının lift kadar artar.


### Adım 5) En son 2_0 hizmetini alan bir kullanıcıya hizmet önerisinde bulununuz.

In [11]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending = False)
    recommendation_list = []
    for i, product in sorted_rules["antecedents"].items():
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"]))
    recommendation_list = list({item for item_list in recommendation_list for item in item_list})
    return recommendation_list[:rec_count]

arl_recommender(rules, "2_0",2)

['15_1', '22_0']